In [ ]:
from arch import arch_model
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
AAPL_data = pd.read_csv('data/AAPL.csv')
AAPL_data = AAPL_data.set_index('Date')

#인덱스를 datetime으로 변경
AAPL_data.index = pd.to_datetime(AAPL_data.index)
AAPL_data['log_price_return'] = np.log(AAPL_data['Adj Close']).diff(1) 
AAPL_data['squared_log_price_return'] = AAPL_data['log_price_return'] ** 2

#library에서 scale을 위해 100을 곱해서 사용
returns = AAPL_data['log_price_return'].dropna()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
volatility = AAPL_data['squared_log_price_return'].dropna().rolling(window=4).sum() / 5.0
normalzied_volatility = scaler.fit_transform(volatility.values.reshape(-1,1))


In [ ]:
import numpy as np
from scipy.optimize import differential_evolution, LinearConstraint
import arch

# GARCH(1,1) 모델의 로그 우도 함수 정의
def garch_log_likelihood(params, returns):
    omega, alpha, beta = params
    model = arch.arch_model(returns, vol='Garch',mean='Zero', p=1, q=1)
    results = model.fit(disp='normal', show_warning=False, options={'maxiter': 0},
                        starting_values=np.array([omega, alpha, beta]))
    
    log_likelihood = results.loglikelihood

    print(log_likelihood)
    print(params)
    
    return -log_likelihood

returns = AAPL_data['log_price_return'].dropna() * 100 #scaling 
train_data= returns[:2999,]

# 최적화 범위 설정
bounds = [(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]
#조건 alpha + beta < 1
LC = LinearConstraint([[0,1,1]], lb=0, ub=1)

# DE 알고리즘을 사용한 파라미터 추정
# minimizing garch -log_likelihood 
result = differential_evolution(garch_log_likelihood, bounds=bounds, args=(train_data,), constraints=LC,
                                strategy='best1bin', maxiter=30, popsize=20, tol=1e-4, mutation=(0.4, 1),
                                recombination=0.7)


print("Optimized GARCH(1,1) Parameters:")
print(result.x)


In [ ]:
omega, alpha, beta = [0.0016,0.32,0.67]
# omega, alpha, beta = [0.10804462 ,0.1053612  ,0.8944112 ]

model = arch.arch_model(returns, vol='Garch',mean='Zero', p=1, q=1)
results = model.fit(disp='off', show_warning=False, options={'maxiter': 0},
                    starting_values=np.array([omega, alpha, beta]))
log_likelihood = results.loglikelihood
-log_likelihood
results.params

In [ ]:
optimal_params = result.x

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
#train 까지 fit하고 나머지 예측
returns = AAPL_data['log_price_return'].dropna() 

start = dt.datetime(2000,1,4)
end = dt.datetime(2013,11,29)
split_date = dt.datetime(2011,12,2)

index = returns.index
start_loc = 0
end_loc = np.where(index >= split_date)[0].min()

AAPL_garch = arch_model(returns, vol='Garch', p=1, o = 0, q = 1, dist='Normal')
fixes = AAPL_garch.fix([1.20539592e-05 ,1.04978855e-01, 8.92015168e-01])
# fixes = AAPL_garch.fix([0.0016,0.32,0.67])
forecasts = fixes.forecast(start = end_loc)
plt.plot(forecasts.variance.dropna())

forecasts.variance.dropna()

In [ ]:
AAPL_data = pd.read_csv('data/AAPL.csv')
AAPL_data['log_price_return'] = np.log(AAPL_data['Adj Close']).diff(1)
AAPL_data['squared_log_price_return'] = AAPL_data['log_price_return'] ** 2
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
#논문에서 명시된 realized volatility 
volatility = AAPL_data['squared_log_price_return'].dropna().rolling(window=5).sum() / 5.0

ret = AAPL_data['squared_log_price_return'].dropna() 

plt.figure(figsize=(10,6))
#volatility 정규화
scaler = MinMaxScaler()
normalzied_volatility = scaler.fit_transform(volatility.values.reshape(-1,1))
plt.plot(normalzied_volatility[2999:])
